In [1]:
import json
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import nltk  # Here to have a nice missing dependency error message early on
from nltk.stem import *
import numpy as np
from datasets import load_dataset, load_metric

from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed, TrainingArguments, TrainerState, TrainerControl,
    TrainerCallback
)

logger = logging.getLogger(__name__)

try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)


2022-03-06 00:55:47.260454: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )
    resize_position_embeddings: Optional[bool] = field(
        default=None,
        metadata={
            "help": "Whether to automatically resize the position embeddings if `max_source_length` exceeds "
            "the model's position embeddings."
        },
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    lang: str = field(default=None, metadata={"help": "Language id for summarization."})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    text_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the full texts (for summarization)."},
    )
    keyphrase_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the summaries (for summarization)."},
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines or csv file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the metrics (rouge) on "
            "(a jsonlines or csv file)."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input test data file to evaluate the metrics (rouge) on " "(a jsonlines or csv file)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=1024,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    max_target_length: Optional[int] = field(
        default=128,
        metadata={
            "help": "The maximum total sequence length for target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total sequence length for validation target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
            "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
            "during ``evaluate`` and ``predict``."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
            "which is used during ``evaluate`` and ``predict``."
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default="", metadata={"help": "A prefix to add before every source text (useful for T5 models)."}
    )

    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": "The token to force as the first generated token after the decoder_start_token_id."
            "Useful for multilingual models like mBART where the first generated token"
            "needs to be the target language token (Usually it is the target language token)"
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length



In [3]:
model_name_or_path = 'memray/bart_wikikp'
cache_dir = './hf_cache'
dataset_name='midas/duc2001'
num_beams=1
max_length=128
max_target_length=128
padding='max_length'
prefix='<present>10<header>5<category>5<seealso>2<infill>0<s>'
# Get the column names for input/target.
text_column = 'document'
keyphrase_column = 'extractive_keyphrases'

training_args = Seq2SeqTrainingArguments(per_device_eval_batch_size=8, output_dir=cache_dir)

In [14]:
sys.argv = ['wikibart_inference.ipynb'] +\
('--config_name memray/bart_wikikp --model_name_or_path memray/bart_wikikp --tokenizer_name memray/bart_wikikp --dataset_name midas/duc2001 --do_predict --output_dir kp_output/duc2001/ --overwrite_output_dir --per_device_eval_batch_size 32 --predict_with_generate --text_column document --keyphrase_column extractive_keyphrases --source_prefix <present>10<header>5<category>5<seealso>2<infill>0<s> --num_beams 1 --generation_max_length 60'.split())
print(sys.argv)

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, Seq2SeqTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


['wikibart_inference.ipynb', '--config_name', 'memray/bart_wikikp', '--model_name_or_path', 'memray/bart_wikikp', '--tokenizer_name', 'memray/bart_wikikp', '--dataset_name', 'midas/duc2001', '--do_predict', '--output_dir', 'kp_output/duc2001/', '--overwrite_output_dir', '--per_device_eval_batch_size', '32', '--predict_with_generate', '--text_column', 'document', '--keyphrase_column', 'extractive_keyphrases', '--source_prefix', '<present>10<header>5<category>5<seealso>2<infill>0<s>', '--num_beams', '1', '--generation_max_length', '60']


In [5]:
config = AutoConfig.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)

model.resize_token_embeddings(len(tokenizer))

if (
        hasattr(model.config, "max_position_embeddings")
        and model.config.max_position_embeddings < data_args.max_source_length
    ):
        if model_args.resize_position_embeddings is None:
            logger.warning(
                f"Increasing the model's number of position embedding vectors from {model.config.max_position_embeddings} "
                f"to {data_args.max_source_length}."
            )
            model.resize_position_embeddings(data_args.max_source_length)
        elif model_args.resize_position_embeddings:
            model.resize_position_embeddings(data_args.max_source_length)
        else:
            raise ValueError(
                f"`--max_source_length` is set to {data_args.max_source_length}, but the model only has {model.config.max_position_embeddings}"
                f" position encodings. Consider either reducing `--max_source_length` to {model.config.max_position_embeddings} or to automatically "
                "resize the model's position encodings by passing `--resize_position_embeddings`."
            )


In [11]:
def preprocess_function(examples):
    # remove pairs where at least one record is None
    inputs, targets = [], []
    for i in range(len(examples[text_column])):
        if examples[text_column][i] is not None and examples[keyphrase_column][i] is not None:
            inputs.append(examples[text_column][i])
            targets.append(examples[keyphrase_column][i])

    inputs = examples[text_column]
    targets = ['<sep>'.join(kps) for kps in examples[keyphrase_column]]
    inputs = [prefix + ' '.join(inp) for inp in inputs]
    model_inputs = tokenizer(inputs, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Metric
def postprocess_text(preds, labels, sep_token):
    stemmer = PorterStemmer()
    preds = [pred.lower().replace('</s>', '').replace('<pad>', '').split(sep_token) for pred in preds]
    labels = [label.lower().replace('<s>', '').replace('<pad>', '').split(sep_token) for label in labels]
    preds = [[' '.join([stemmer.stem(w) for w in p.split()]) for p in pred] for pred in preds]
    labels = [[' '.join([stemmer.stem(w) for w in p.split()]) for p in label] for label in labels]
    preds = [[p.strip() for p in pred if len(p.strip()) > 0] for pred in preds]
    labels = [[p.strip() for p in label if len(p.strip()) > 0] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    if isinstance(preds, tuple):
        preds = preds[0]
    print(preds.shape)
    if len(preds.shape) == 3:
        preds = preds.argmax(axis=-1)
    
    raw_decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=False)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(raw_decoded_preds, decoded_labels, tokenizer.sep_token)

    precs, recalls, f_scores = [], [], []
    num_match, num_pred, num_gold = [], [], []
    for raw_pred, pred, label in zip(raw_decoded_preds, decoded_preds, decoded_labels):
        pred_set = set(pred)
        label_set = set(label)
        match_set = label_set.intersection(pred_set)
        p = float(len(match_set)) / float(len(pred_set)) if len(pred_set) > 0 else 0.0
        r = float(len(match_set)) / float(len(label_set)) if len(label_set) > 0 else 0.0
        f1 = float(2 * (p * r)) / (p + r) if (p + r) > 0 else 0.0
        precs.append(p)
        recalls.append(r)
        f_scores.append(f1)
        num_match.append(len(match_set))
        num_pred.append(len(pred_set))
        num_gold.append(len(label_set))
        
#         print(f'raw_PRED: {raw_pred}')
        print(f'PRED: num={len(pred_set)} - {pred_set}')
        print(f'GT: num={len(label_set)} - {label_set}')
        print(f'p={p}, r={r}, f1={f1}')
        print('-' * 20)

    result = {
        'precision@M': np.mean(precs) * 100.0,
        'recall@M': np.mean(recalls) * 100.0,
        'fscore@M': np.mean(f_scores) * 100.0,
        'num_match': np.mean(num_match),
        'num_pred': np.mean(num_pred),
        'num_gold': np.mean(num_gold),
    }

    result = {k: round(v, 4) for k, v in result.items()}
    return result


In [15]:
# Set seed before initializing model.
set_seed(training_args.seed)
raw_datasets = load_dataset(
    dataset_name, "raw", cache_dir=cache_dir
)

column_names = raw_datasets["test"].column_names
max_target_length = data_args.val_max_target_length
padding = "max_length" if data_args.pad_to_max_length else False
prefix = data_args.source_prefix if data_args.source_prefix is not None else ""
predict_dataset = raw_datasets["test"]

with training_args.main_process_first(desc="prediction dataset map pre-processing"):
    predict_dataset = predict_dataset.map(
        preprocess_function,
        batched=True,
        num_proc=data_args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not data_args.overwrite_cache,
        desc="Running tokenizer on prediction dataset",
    )

# Data collator
label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if training_args.fp16 else None,
)

Reusing dataset duc2001 (./hf_cache/midas___duc2001/raw/0.0.1/7888b46165d8a58f49f00e28410b46b1f22fabfd72a9e89f3e80a4e2d27e4a9b)


  0%|          | 0/1 [00:00<?, ?it/s]

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name())
# # device = torch.device("cuda:2")
# torch.cuda.set_device(1)
# device = torch.device()
# print(device)
print(torch.cuda.device_count())

model = model.to('cuda:0')

1


In [16]:
# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if training_args.predict_with_generate else None,
)

# Evaluation
results = {}
predict_results = trainer.predict(
    predict_dataset, metric_key_prefix="predict", max_length=max_length, num_beams=num_beams,
)
metrics = predict_results.metrics
max_predict_samples = len(predict_dataset)
metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

trainer.log_metrics("predict", metrics)
trainer.save_metrics("predict", metrics)

if trainer.is_world_process_zero():
    if training_args.predict_with_generate:
        predictions = tokenizer.batch_decode(predict_results.predictions)
        predictions = [pred.lower().replace('</s>', '').replace('<pad>', '').strip().split(tokenizer.sep_token) for pred in predictions]
        output_prediction_file = os.path.join(training_args.output_dir, "generated_predictions.txt")
        with open(output_prediction_file, "w") as writer:
            writer.write("\n".join([json.dumps(pred) for pred in predictions]))

kwargs = {"finetuned_from": model_name_or_path, "tasks": "keyphrasification"}


***** Running Prediction *****
  Num examples = 308
  Batch size = 32


(308, 128)
PRED: num=26 - {'panamerican world airway', 'aardvark', 'and the follow develop today:', 'list of aircraft', 'articl contain video clip', 'and', 'lockerbi', 'scotland, scotland', 'scotland', 'aviation-rel death', 'histori of aviat', 'aerojet aircraft', 'airbu a380', 'aeroship aircraft', 'pan american world airway flight 103', 'pan american', 'histori', 'scotland,', 'larnockerbi flight 103 flight 103 (wednesday night)', 'aerial aircraft', 'aircraft accid', 'panamer', 'lancasterbi', 'aerospac accid', 'airlin', 'aero'}
GT: num=3 - {'pan american world airway flight 103', 'lockerbi', 'crash'}
p=0.07692307692307693, r=0.6666666666666666, f1=0.13793103448275862
--------------------
PRED: num=18 - {'cultur of the southern unit state', 'bunki', 'buddi mcintyr', 'texa', 'tunnel alley', 'american geographi', 'histori of the unit state (1825–1921)', 'historyof the unit state (1921–present)', 'tropic storm', 'and a', 'american weather', 'and', 'histori', 't tornado alley', 'geographi of

In [10]:
import transformers
transformers.__version__

'4.17.0'